# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
!pip install cartopy
!pip install geoviews

  Obtaining dependency information for cartopy from https://files.pythonhosted.org/packages/bb/50/ab5495dcc22587179134cd37e6534212babeda78e86bda311c37c773f086/Cartopy-0.22.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for shapely>=1.7 from https://files.pythonhosted.org/packages/9e/39/029c441d8af32ab423b229c4525ce5ce6707318155b59634811a4c56f5c4/shapely-2.0.2-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/46.5 kB ? eta -:--:--
     -------- ------------------------------- 10.2/46.5 kB ? eta -:--:--
     -------------------------------------- 46.5/46.5 kB 772.3 kB/s eta 0:00:00
  Obtaining dependency information for pyproj>=3.1.0 from https://files.pythonhosted.org/packages/79/95/eb68113c5b5737c342bde1bab92705dabe69c16299c5a122616e50f1fbd6/pyproj-3.6.1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/11.2 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.2 MB 3.8 MB/s eta 0

In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
from cartopy import crs

# Import API key
from api_keys import geoapify_key

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

# Configure the map plot
geo_df = gv.Dataset(city_data_df, kdims=['Lng', 'Lat'])

map_plot = gv.Points(geo_df, kdims=['Lng', 'Lat']).opts(color='blue', size=15, tools=['hover'], alpha=0.7, cmap='viridis')

# Display the map
map_plot

:Points   [Lng,Lat]   (City_ID,City,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [32]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[(city_data_df['Max Temp'] <60) & (city_data_df['Max Temp']>15) & 
                               (city_data_df['Wind Speed'] < 3.00) & (city_data_df['Cloudiness'] == 0)
                              ]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
51,51,hilo,19.7297,-155.0900,26.27,83,0,2.57,US,1666108260
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
81,81,makakilo city,21.3469,-158.0858,21.66,81,0,2.57,US,1666108282
128,128,half moon bay,37.4636,-122.4286,19.71,86,0,0.00,US,1666108312
211,211,laguna,38.4210,-121.4238,21.67,79,0,2.06,US,1666108364


### Step 3: Create a new DataFrame called `hotel_df`.

In [33]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
51,hilo,US,19.7297,-155.0900,83,
63,banda,IN,25.4833,80.3333,52,
81,makakilo city,US,21.3469,-158.0858,81,
128,half moon bay,US,37.4636,-122.4286,86,
211,laguna,US,38.4210,-121.4238,79,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [35]:
# Set parameters to search for a hotel
radius = 20000
params = {
    'categories': 'lodging',
    'radius': radius,
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle: {lon}, {lat}, {radius}"
    params["bias"] = f'proximity: {lon}, {lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hilo - nearest hotel: No hotel found
banda - nearest hotel: No hotel found
makakilo city - nearest hotel: No hotel found
half moon bay - nearest hotel: No hotel found
laguna - nearest hotel: No hotel found
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: No hotel found
santa rosalia - nearest hotel: No hotel found
narwar - nearest hotel: No hotel found
roebourne - nearest hotel: No hotel found
stolac - nearest hotel: No hotel found
lakki marwat - nearest hotel: No hotel found
san joaquin - nearest hotel: No hotel found
pleasanton - nearest hotel: No hotel found
tikrit - nearest hotel: No hotel found
guerrero negro - nearest hotel: No hotel found
pali - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
51,hilo,US,19.7297,-155.0900,83,No hotel found
63,banda,IN,25.4833,80.3333,52,No hotel found
81,makakilo city,US,21.3469,-158.0858,81,No hotel found
128,half moon bay,US,37.4636,-122.4286,86,No hotel found
211,laguna,US,38.4210,-121.4238,79,No hotel found
240,tikaitnagar,IN,26.9500,81.5833,59,No hotel found
265,san quintin,MX,30.4833,-115.9500,74,No hotel found
340,santa rosalia,MX,27.3167,-112.2833,56,No hotel found
363,narwar,IN,25.6500,77.9000,55,No hotel found
381,roebourne,AU,-20.7833,117.1333,65,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [36]:
%%capture --no-display

# Configure the map plot
hotel_map_df = gv.Dataset(hotel_df, kdims=['Lng', 'Lat'])

map_plot = gv.Points(hotel_df, kdims=['Lng', 'Lat']).opts(color='blue', size=15, tools=['hover'], alpha=0.7, cmap='viridis')

# Display the map
map_plot

:Points   [Lng,Lat]   (City,Country,Humidity,Hotel Name)